In [53]:
import torch

boxes_nms=torch.randint(-1000,1000,(3,4))
print(boxes_nms)

tensor([[ 388, -949,  355,  564],
        [ 236, -732, -886, -698],
        [ 171, -826,  485, -693]])


In [54]:
boxes_nms[:, 0] = torch.clamp(boxes_nms[:, 0], 10)
boxes_nms[:, 1] = torch.clamp(boxes_nms[:, 1], max=500)
boxes_nms

tensor([[ 388, -949,  355,  564],
        [ 236, -732, -886, -698],
        [ 171, -826,  485, -693]])

In [55]:
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))
torch.manual_seed(42)
X = torch.rand(2, 20)

X, net(X)

(tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904, 0.6009, 0.2566, 0.7936, 0.9408,
          0.1332, 0.9346, 0.5936, 0.8694, 0.5677, 0.7411, 0.4294, 0.8854, 0.5739,
          0.2666, 0.6274],
         [0.2696, 0.4414, 0.2969, 0.8317, 0.1053, 0.2695, 0.3588, 0.1994, 0.5472,
          0.0062, 0.9516, 0.0753, 0.8860, 0.5832, 0.3376, 0.8090, 0.5779, 0.9040,
          0.5547, 0.3423]]),
 tensor([[-0.2252,  0.2366,  0.0591, -0.1059, -0.2802, -0.3032,  0.1990, -0.1906,
          -0.0541, -0.0177],
         [-0.1857,  0.2777,  0.0907, -0.1178, -0.2714, -0.1451,  0.2068, -0.0855,
          -0.1081,  0.0178]], grad_fn=<AddmmBackward0>))

In [56]:
# define a MLP module, implement the same funtion as Net above
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    
net = MLP()
X, net(X)

(tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904, 0.6009, 0.2566, 0.7936, 0.9408,
          0.1332, 0.9346, 0.5936, 0.8694, 0.5677, 0.7411, 0.4294, 0.8854, 0.5739,
          0.2666, 0.6274],
         [0.2696, 0.4414, 0.2969, 0.8317, 0.1053, 0.2695, 0.3588, 0.1994, 0.5472,
          0.0062, 0.9516, 0.0753, 0.8860, 0.5832, 0.3376, 0.8090, 0.5779, 0.9040,
          0.5547, 0.3423]]),
 tensor([[ 0.1074,  0.1122, -0.0164, -0.1884,  0.2030,  0.1430,  0.0029, -0.0170,
           0.0118, -0.3402],
         [ 0.0556,  0.0728,  0.0151, -0.1841,  0.2343,  0.1248,  0.0682,  0.0488,
          -0.0727, -0.1874]], grad_fn=<AddmmBackward0>))

### Parameter managements

In [57]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),
                    nn.ReLU(),
                    nn.Linear(8,1))

X = torch.rand((2,4))
net(X)

tensor([[-0.1784],
        [-0.2106]], grad_fn=<AddmmBackward0>)

In [62]:
# access those parameters in layers
print(net[2].state_dict())


OrderedDict([('weight', tensor([[ 0.0138, -0.2764, -0.1944,  0.2576,  0.0113, -0.1603,  0.2390, -0.1049]])), ('bias', tensor([-0.1438]))])


In [63]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.1438], requires_grad=True)
tensor([-0.1438])


In [71]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

print(*[(name, param.shape) for name, param in net.named_parameters()])


('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [ ]:
print(net[2].state_dict()['bias'].data)
print(net.state_dict()['2.bias'].data) # same as

tensor([-0.1438])
tensor([-0.1438])


In [82]:
def block1():
    return nn.Sequential(nn.Linear(4,8),
                         nn.ReLU(),
                         nn.Linear(8,4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[-0.2327],
        [-0.2327]], grad_fn=<AddmmBackward0>)

In [84]:
print(*[(name, param.shape) for name, param in rgnet.named_parameters()], sep="\n")

('0.block 0.0.weight', torch.Size([8, 4]))
('0.block 0.0.bias', torch.Size([8]))
('0.block 0.2.weight', torch.Size([4, 8]))
('0.block 0.2.bias', torch.Size([4]))
('0.block 1.0.weight', torch.Size([8, 4]))
('0.block 1.0.bias', torch.Size([8]))
('0.block 1.2.weight', torch.Size([4, 8]))
('0.block 1.2.bias', torch.Size([4]))
('0.block 2.0.weight', torch.Size([8, 4]))
('0.block 2.0.bias', torch.Size([8]))
('0.block 2.2.weight', torch.Size([4, 8]))
('0.block 2.2.bias', torch.Size([4]))
('0.block 3.0.weight', torch.Size([8, 4]))
('0.block 3.0.bias', torch.Size([8]))
('0.block 3.2.weight', torch.Size([4, 8]))
('0.block 3.2.bias', torch.Size([4]))
('1.weight', torch.Size([1, 4]))
('1.bias', torch.Size([1]))
